# Transformer

* Reference: https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [1]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from torch.utils.data import dataset

def generate_square_subsequent_mask(sz: int) -> Tensor:
    """
    Generates an upper-triangular matrix of -int, with zeros on dialog
    refer to https://pytorch.org/docs/stable/generated/torch.triu.html
    """
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float=0.1, max_len: int=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        
    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        # here we also add a dropout layer for position encoding
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Tansformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        # should be replaced with TransformerDeCoder
        self.decoder = nn.Linear(d_model, ntoken)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len
            
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [2]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk'])
vocab.set_default_index(vocab['<unk'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """
    converts raw text into a flat tensor
    """
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab, so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """
    Divides the data into bsz separate sequences, removing extra elements that wouldn't clearly fit.
    
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
        
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    # rounding up based on batch size
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    # transform to bsz * seq_len, then use contiguous() to return a contiguous in memory tensor 
    # containing the same data as self tensor. If self tensor is already in the specified memory format, 
    # this function returns the self tensor: refer to https://pytorch.org/docs/stable/generated/torch.Tensor.contiguous.html?highlight=contiguous#torch.Tensor.contiguous
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size) # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [3]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        sources: Tensor, shape [full_seq_len, batch_size]
        i: int
        
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [4]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [5]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [6]:
%%time
best_val_loss = float('inf')
epochs = 1000
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 10.59 | loss  8.19 | ppl  3596.99
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch  9.26 | loss  6.92 | ppl  1009.33
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch  9.20 | loss  6.46 | ppl   640.37
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch  9.23 | loss  6.31 | ppl   550.80
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch  9.22 | loss  6.19 | ppl   488.88
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch  9.21 | loss  6.17 | ppl   475.94
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch  9.22 | loss  6.12 | ppl   453.07
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch  9.20 | loss  6.12 | ppl   455.65
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch  9.25 | loss  6.03 | ppl   415.84
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch  9.21 | loss  6.04 | ppl   418.11
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch  9.21 | loss  5.90 | ppl   365.06
| epoch   1 |  2400/ 

| epoch   6 |  1800/ 2928 batches | lr 3.87 | ms/batch  8.68 | loss  5.12 | ppl   168.16
| epoch   6 |  2000/ 2928 batches | lr 3.87 | ms/batch  8.67 | loss  5.14 | ppl   170.44
| epoch   6 |  2200/ 2928 batches | lr 3.87 | ms/batch  8.59 | loss  4.99 | ppl   147.45
| epoch   6 |  2400/ 2928 batches | lr 3.87 | ms/batch  8.73 | loss  5.11 | ppl   165.40
| epoch   6 |  2600/ 2928 batches | lr 3.87 | ms/batch  8.84 | loss  5.13 | ppl   168.18
| epoch   6 |  2800/ 2928 batches | lr 3.87 | ms/batch  8.77 | loss  5.05 | ppl   156.28
-----------------------------------------------------------------------------------------
| end of epoch   6 | time: 26.35s | valid loss  5.53 | valid ppl   250.91
-----------------------------------------------------------------------------------------
| epoch   7 |   200/ 2928 batches | lr 3.68 | ms/batch  9.50 | loss  5.10 | ppl   164.41
| epoch   7 |   400/ 2928 batches | lr 3.68 | ms/batch  8.68 | loss  5.14 | ppl   169.92
| epoch   7 |   600/ 2928 batches 

| epoch  12 |   200/ 2928 batches | lr 2.84 | ms/batch  8.88 | loss  4.77 | ppl   117.36
| epoch  12 |   400/ 2928 batches | lr 2.84 | ms/batch  8.41 | loss  4.79 | ppl   120.46
| epoch  12 |   600/ 2928 batches | lr 2.84 | ms/batch  8.40 | loss  4.61 | ppl   100.97
| epoch  12 |   800/ 2928 batches | lr 2.84 | ms/batch  8.40 | loss  4.67 | ppl   107.13
| epoch  12 |  1000/ 2928 batches | lr 2.84 | ms/batch  8.44 | loss  4.67 | ppl   106.50
| epoch  12 |  1200/ 2928 batches | lr 2.84 | ms/batch  8.43 | loss  4.71 | ppl   110.84
| epoch  12 |  1400/ 2928 batches | lr 2.84 | ms/batch  8.49 | loss  4.72 | ppl   111.71
| epoch  12 |  1600/ 2928 batches | lr 2.84 | ms/batch  8.49 | loss  4.77 | ppl   118.38
| epoch  12 |  1800/ 2928 batches | lr 2.84 | ms/batch  8.52 | loss  4.72 | ppl   112.59
| epoch  12 |  2000/ 2928 batches | lr 2.84 | ms/batch 11.44 | loss  4.73 | ppl   113.00
| epoch  12 |  2200/ 2928 batches | lr 2.84 | ms/batch 11.46 | loss  4.59 | ppl    98.01
| epoch  12 |  2400/ 

| epoch  17 |  1800/ 2928 batches | lr 2.20 | ms/batch  8.35 | loss  4.52 | ppl    91.60
| epoch  17 |  2000/ 2928 batches | lr 2.20 | ms/batch  8.35 | loss  4.51 | ppl    90.66
| epoch  17 |  2200/ 2928 batches | lr 2.20 | ms/batch  8.33 | loss  4.37 | ppl    78.94
| epoch  17 |  2400/ 2928 batches | lr 2.20 | ms/batch  8.33 | loss  4.47 | ppl    87.54
| epoch  17 |  2600/ 2928 batches | lr 2.20 | ms/batch  8.40 | loss  4.49 | ppl    89.47
| epoch  17 |  2800/ 2928 batches | lr 2.20 | ms/batch  8.33 | loss  4.45 | ppl    85.20
-----------------------------------------------------------------------------------------
| end of epoch  17 | time: 25.79s | valid loss  5.56 | valid ppl   260.34
-----------------------------------------------------------------------------------------
| epoch  18 |   200/ 2928 batches | lr 2.09 | ms/batch  8.46 | loss  4.50 | ppl    90.27
| epoch  18 |   400/ 2928 batches | lr 2.09 | ms/batch  8.32 | loss  4.52 | ppl    91.99
| epoch  18 |   600/ 2928 batches 

| epoch  23 |   200/ 2928 batches | lr 1.62 | ms/batch  8.51 | loss  4.35 | ppl    77.42
| epoch  23 |   400/ 2928 batches | lr 1.62 | ms/batch  8.33 | loss  4.37 | ppl    78.90
| epoch  23 |   600/ 2928 batches | lr 1.62 | ms/batch  8.34 | loss  4.21 | ppl    67.41
| epoch  23 |   800/ 2928 batches | lr 1.62 | ms/batch  8.44 | loss  4.27 | ppl    71.74
| epoch  23 |  1000/ 2928 batches | lr 1.62 | ms/batch  8.46 | loss  4.28 | ppl    72.19
| epoch  23 |  1200/ 2928 batches | lr 1.62 | ms/batch  8.37 | loss  4.31 | ppl    74.69
| epoch  23 |  1400/ 2928 batches | lr 1.62 | ms/batch  8.34 | loss  4.31 | ppl    74.32
| epoch  23 |  1600/ 2928 batches | lr 1.62 | ms/batch  8.34 | loss  4.36 | ppl    77.99
| epoch  23 |  1800/ 2928 batches | lr 1.62 | ms/batch  8.34 | loss  4.34 | ppl    76.53
| epoch  23 |  2000/ 2928 batches | lr 1.62 | ms/batch  8.36 | loss  4.32 | ppl    75.50
| epoch  23 |  2200/ 2928 batches | lr 1.62 | ms/batch  8.35 | loss  4.18 | ppl    65.39
| epoch  23 |  2400/ 

...

| epoch 974 |  1800/ 2928 batches | lr 0.00 | ms/batch  8.36 | loss  4.05 | ppl    57.61
| epoch 974 |  2000/ 2928 batches | lr 0.00 | ms/batch  8.35 | loss  4.05 | ppl    57.49
| epoch 974 |  2200/ 2928 batches | lr 0.00 | ms/batch  8.35 | loss  3.91 | ppl    49.70
| epoch 974 |  2400/ 2928 batches | lr 0.00 | ms/batch  8.36 | loss  4.00 | ppl    54.50
| epoch 974 |  2600/ 2928 batches | lr 0.00 | ms/batch  8.34 | loss  4.02 | ppl    55.60
| epoch 974 |  2800/ 2928 batches | lr 0.00 | ms/batch  8.33 | loss  3.99 | ppl    54.27
-----------------------------------------------------------------------------------------
| end of epoch 974 | time: 25.83s | valid loss  5.54 | valid ppl   253.63
-----------------------------------------------------------------------------------------
| epoch 975 |   200/ 2928 batches | lr 0.00 | ms/batch  8.41 | loss  4.07 | ppl    58.64
| epoch 975 |   400/ 2928 batches | lr 0.00 | ms/batch  8.37 | loss  4.08 | ppl    59.04
| epoch 975 |   600/ 2928 batches 

| epoch 980 |   200/ 2928 batches | lr 0.00 | ms/batch  8.45 | loss  4.07 | ppl    58.63
| epoch 980 |   400/ 2928 batches | lr 0.00 | ms/batch  8.30 | loss  4.07 | ppl    58.73
| epoch 980 |   600/ 2928 batches | lr 0.00 | ms/batch  8.30 | loss  3.94 | ppl    51.27
| epoch 980 |   800/ 2928 batches | lr 0.00 | ms/batch  8.29 | loss  4.00 | ppl    54.81
| epoch 980 |  1000/ 2928 batches | lr 0.00 | ms/batch  8.29 | loss  4.01 | ppl    55.21
| epoch 980 |  1200/ 2928 batches | lr 0.00 | ms/batch  8.31 | loss  4.04 | ppl    57.01
| epoch 980 |  1400/ 2928 batches | lr 0.00 | ms/batch  8.33 | loss  4.01 | ppl    55.41
| epoch 980 |  1600/ 2928 batches | lr 0.00 | ms/batch  8.34 | loss  4.07 | ppl    58.47
| epoch 980 |  1800/ 2928 batches | lr 0.00 | ms/batch  8.31 | loss  4.06 | ppl    57.84
| epoch 980 |  2000/ 2928 batches | lr 0.00 | ms/batch  8.33 | loss  4.05 | ppl    57.60
| epoch 980 |  2200/ 2928 batches | lr 0.00 | ms/batch  8.33 | loss  3.90 | ppl    49.60
| epoch 980 |  2400/ 

| epoch 985 |  1800/ 2928 batches | lr 0.00 | ms/batch  8.48 | loss  4.05 | ppl    57.62
| epoch 985 |  2000/ 2928 batches | lr 0.00 | ms/batch  8.41 | loss  4.05 | ppl    57.54
| epoch 985 |  2200/ 2928 batches | lr 0.00 | ms/batch  8.42 | loss  3.91 | ppl    49.77
| epoch 985 |  2400/ 2928 batches | lr 0.00 | ms/batch  8.86 | loss  4.00 | ppl    54.60
| epoch 985 |  2600/ 2928 batches | lr 0.00 | ms/batch 10.08 | loss  4.02 | ppl    55.75
| epoch 985 |  2800/ 2928 batches | lr 0.00 | ms/batch 10.11 | loss  3.99 | ppl    54.08
-----------------------------------------------------------------------------------------
| end of epoch 985 | time: 27.00s | valid loss  5.54 | valid ppl   253.63
-----------------------------------------------------------------------------------------
| epoch 986 |   200/ 2928 batches | lr 0.00 | ms/batch  8.56 | loss  4.07 | ppl    58.75
| epoch 986 |   400/ 2928 batches | lr 0.00 | ms/batch  8.42 | loss  4.07 | ppl    58.58
| epoch 986 |   600/ 2928 batches 

| epoch 991 |   200/ 2928 batches | lr 0.00 | ms/batch  9.32 | loss  4.07 | ppl    58.31
| epoch 991 |   400/ 2928 batches | lr 0.00 | ms/batch  9.14 | loss  4.08 | ppl    58.86
| epoch 991 |   600/ 2928 batches | lr 0.00 | ms/batch  9.22 | loss  3.94 | ppl    51.58
| epoch 991 |   800/ 2928 batches | lr 0.00 | ms/batch  9.23 | loss  4.00 | ppl    54.82
| epoch 991 |  1000/ 2928 batches | lr 0.00 | ms/batch  9.43 | loss  4.01 | ppl    55.29
| epoch 991 |  1200/ 2928 batches | lr 0.00 | ms/batch 10.01 | loss  4.04 | ppl    56.82
| epoch 991 |  1400/ 2928 batches | lr 0.00 | ms/batch 10.00 | loss  4.01 | ppl    55.28
| epoch 991 |  1600/ 2928 batches | lr 0.00 | ms/batch 10.06 | loss  4.07 | ppl    58.75
| epoch 991 |  1800/ 2928 batches | lr 0.00 | ms/batch 10.01 | loss  4.05 | ppl    57.61
| epoch 991 |  2000/ 2928 batches | lr 0.00 | ms/batch 10.02 | loss  4.06 | ppl    57.75
| epoch 991 |  2200/ 2928 batches | lr 0.00 | ms/batch 10.06 | loss  3.91 | ppl    49.74
| epoch 991 |  2400/ 

| epoch 996 |  1800/ 2928 batches | lr 0.00 | ms/batch  8.48 | loss  4.05 | ppl    57.66
| epoch 996 |  2000/ 2928 batches | lr 0.00 | ms/batch  8.45 | loss  4.05 | ppl    57.61
| epoch 996 |  2200/ 2928 batches | lr 0.00 | ms/batch  8.44 | loss  3.90 | ppl    49.64
| epoch 996 |  2400/ 2928 batches | lr 0.00 | ms/batch  8.48 | loss  4.00 | ppl    54.50
| epoch 996 |  2600/ 2928 batches | lr 0.00 | ms/batch  8.44 | loss  4.03 | ppl    55.99
| epoch 996 |  2800/ 2928 batches | lr 0.00 | ms/batch  8.43 | loss  3.99 | ppl    54.04
-----------------------------------------------------------------------------------------
| end of epoch 996 | time: 26.17s | valid loss  5.54 | valid ppl   253.63
-----------------------------------------------------------------------------------------
| epoch 997 |   200/ 2928 batches | lr 0.00 | ms/batch  8.54 | loss  4.07 | ppl    58.67
| epoch 997 |   400/ 2928 batches | lr 0.00 | ms/batch  8.37 | loss  4.08 | ppl    58.85
| epoch 997 |   600/ 2928 batches 